In [1]:
try:
    # from google.colab import drive
    # drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/LisaDataset.zip', 'r') as zip_ref:
        zip_ref.extractall('./LisaDataset')
except:
    print("Using Local Machine")
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

Using Local Machine
Cloning into 'yolov5'...
remote: Enumerating objects: 16088, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 16088 (delta 10), reused 14 (delta 2), pack-reused 16056
Receiving objects: 100% (16088/16088), 14.71 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (11038/11038), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.1/660.1 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

In [2]:
# Include all packages
import os
import cv2
from time import time
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
from yolov5.models.yolo import Model
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import torchvision


In [3]:

def imgResize(img, x1, y1, x2, y2, newW, newH):
    origHeight, origWidth = img.shape[:2]
    scaleWidth = newW / origWidth
    scaleHeight = newH / origHeight
    imgResized = cv2.resize(
        img, (newW, newH), interpolation=cv2.INTER_LINEAR)
    newX1, newy1 = int(x1 * scaleWidth), int(y1 * scaleHeight)
    newX2, newy2 = int(x2 * scaleWidth), int(y2 * scaleHeight)
    return imgResized, newX1, newy1, newX2, newy2


In [4]:
def dsLoad(dsFolderPath):
    imgs = []
    annotations = []
    filePathAnnotation = dsFolderPath+"/allAnnotations.csv"
    dfAnnonation = pd.read_csv(filePathAnnotation, sep=";")
    signsUnique = dfAnnonation['Annotation tag'].unique().tolist()
    for index, row in dfAnnonation[1:].iterrows():
        img = cv2.imread(dsFolderPath+"/"+row[0])
        imgs.append(img)
        annotations.append(
            [signsUnique.index(row[1]), row[2], row[3], row[4], row[5]])

    del dfAnnonation

    return imgs, annotations, len(signsUnique)


In [5]:
def dataPreProcess(imgs, annotations, batchSize, resize):
    resizedImgs = []
    newAnnotations = []
    for i, img in enumerate(imgs):
        [label, x1, y1, x2, y2] = annotations[i]
        imgResized, newX1, newy1, newX2, newy2 = imgResize(
            img, x1, y1, x2, y2, resize[0], resize[1])
        resizedImgs.append(imgResized)
        newAnnotations.append(
            [(i % batchSize), label, newX1, newy1, newX2, newy2])

    X_train, X_val, y_train, y_val = train_test_split(
        resizedImgs, newAnnotations, test_size=0.3, random_state=42)

    return X_train, X_val, y_train, y_val


In [6]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label).float()
        return inputData, label


In [7]:
def dataLoadersCreation(X_train, X_val, y_train, y_val, batchSize):
    dsTrain = []
    dsVal = []
    for i in range(len(X_train)):
        dsTrain.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        dsVal.append((X_val[i], y_val[i]))

    dsTrain = CustomDataset(dsTrain)
    dsVal = CustomDataset(dsVal)
    DLTrain = DataLoader(
        dsTrain, batchSize=batchSize, shuffle=True, num_workers=4)
    DLVal = DataLoader(
        dsVal, batchSize=batchSize, shuffle=False, num_workers=4)

    return DLTrain, DLVal


In [8]:

def TargetToTensor(targets, batchSize, noAnchors, gridSizes, noClasses):
    objTarget = []
    objClass = []
    boxTarget = []

    for gridSize in gridSizes:
        objTarget.append(torch.zeros(
            (batchSize, noAnchors, gridSize, gridSize, 1)))
        objClass.append(torch.zeros(
            (batchSize, noAnchors, gridSize, gridSize, noClasses)))
        boxTarget.append(torch.zeros(
            (batchSize, noAnchors, gridSize, gridSize, 4)))

    for target in targets:
        batchIndex, cls, xCenter, yCenter, width, height = target.long()

        for i, gridSize in enumerate(gridSizes):

            xCell, yCell = int(
                xCenter * gridSize), int(yCenter * gridSize)
            anchor = 0
            try:
                objTarget[i][batchIndex, anchor, yCell, xCell, 0] = 1
                objClass[i][batchIndex, anchor, yCell, xCell, cls] = 1
                boxTarget[i][batchIndex, anchor, yCell, xCell] = torch.tensor(
                    [xCenter, yCenter, width, height])
            except Exception as e:
                pass
    return objTarget, objClass, boxTarget


In [9]:
class YOLOv5Loss(nn.Module):
    def __init__(self, noClasses, noAnchors=3):
        super(YOLOv5Loss, self).__init__()
        self.noClasses = noClasses
        self.noAnchors = noAnchors

    def forward(self, preds, targets):
        lossobj = torch.tensor(0.0, device=preds[0].device)
        lossClass = torch.tensor(0.0, device=preds[0].device)
        lossBox = torch.tensor(0.0, device=preds[0].device)
        batchSize = preds[0].size(0)
        gridSizes = [pred.size(2) for pred in preds]
        targetListObj, targetListClass, targetListBox = TargetToTensor(
            targets, batchSize, self.noAnchors, gridSizes, self.noClasses)

        for i, pred in enumerate(preds):
            objTarget = targetListObj[i].to(pred.device)
            objClass = targetListClass[i].to(pred.device)
            boxTarget = targetListBox[i].to(pred.device)

            lossobj += nn.BCEWithLogitsLoss()(pred[..., 4:5], objTarget)
            lossClass += nn.BCEWithLogitsLoss()(pred[..., 5:], objClass)
            lossBox += nn.MSELoss()(pred[..., :4], boxTarget)

        lossTotal = lossobj + lossClass + lossBox
        return lossTotal


In [10]:
def yolov5ModelCreation(noClasses, version="s"):
    configureFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(configureFile, ch=3, nc=noClasses)
    return model


In [11]:
def TrainModel(model, dl, epochs, optimizer, lossFunction, device):
    model.train()
    for epoch in range(epochs):
        print("Epoch {}/{}:".format(epoch+1, epochs))
        st = time()
        lossTotal = 0
        dLLen = len(dl)
        for i, (inputs, targets) in enumerate(dl):
            inputs = inputs.permute(0, 3, 1, 2)
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = lossFunction(outputs, targets)
                loss.backward()
                optimizer.step()

            lossTotal += loss.item() * inputs.size(0)
            if(((i*100)//dLLen) % 10 == 0):
                print((i*100//dLLen), end="%,")

        et = time()
        timeTaken = et-st
        epochLoss = lossTotal / dLLen
        print("Training Loss: {:.4f}".format(epochLoss))
        print("Time taken: {}min, {}, secs".format(timeTaken//60, timeTaken % 60))

    print("Training complete.")
    return model


In [12]:
def ProcessYoloOutput(outputs, shapeOfInput, conf_threshold=0.5, nms_threshold=0.5):
    
    predictions = []
    for output in outputs:
        # perform non-maximum suppression
        box = output[:, :4]
        scores = output[:, 4]
        mask = scores >= conf_threshold
        box = box[mask]
        scores = scores[mask]
        keep = torchvision.ops.box.batched_nms(
            box, scores, torch.zeros_like(scores), nms_threshold)
        box = box[keep]
        scores = scores[keep]

        box[:, [0, 2]] *= shapeOfInput[0]
        box[:, [1, 3]] *= shapeOfInput[1]
        box[:, [0, 2]] = box[:, [0, 2]].clamp(0, shapeOfInput[0] - 1)
        box[:, [1, 3]] = box[:, [1, 3]].clamp(0, shapeOfInput[1] - 1)
        box = box[:, [1, 0, 3, 2]]
        conf = scores
        for box, score in zip(box, conf):
            predictions.append(
                {'bbox': box.tolist(), 'score': score.item(), 'category_id': 1})

    return predictions


def mAPScore(preds, labels):
    coco_gt = COCO()
    for i, (image, box) in enumerate(labels):
        for box in box:
            x1, y1, x2, y2 = box
            w, h = x2 - x1, y2 - y1
            coco_gt.add_annotation({
                'id': len(coco_gt.dataset['annotations']),
                'image_id': i,
                'category_id': 1,
                'bbox': [x1, y1, w, h],
                'area': w * h,
                'iscrowd': 0,
            })

    # create a COCO-format predictions file
    coco_dt = coco_gt.loadRes(preds)

    # compute mAP using COCOeval
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return coco_eval.stats[0]

def EvaluateModel(model, dataLoader, device):
    print("Evaluateing Model:")
    st = time()
    dLLen = len(dataLoader)
    mAPScore = 0
    for i, (input, targets) in enumerate(dataLoader):
        try:
            input = input.permute(0, 3, 1, 2)
            input = input.to(device)
            targets = targets.to(device)
            with torch.no_grad():
                outputs = model(input)
                preds = ProcessYoloOutput(outputs, input.shape)
                print(preds)
                mAPScore = mAPScore(preds, targets)
        except:
            pass
        if(((i*100)//dLLen) % 10 == 0):
            print((i*100//dLLen), end="%,")

    et = time()
    timeTaken = et-st
    print("mAP score on validation set: {:.4f}".format(mAPScore))
    print("Time taken: {}min, {}, secs".format(timeTaken//60, timeTaken % 60))


In [13]:
batchSize = 32
shapeOfInput = (416, 416)
epochs = 100
noAnchors = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
print("Using {} device".format(device))

Using cuda device


In [15]:
imgs, annotations, noClasses = dsLoad("/content/drive/MyDrive/LisaDataset")


In [16]:
X_train, X_val, y_train, y_val = dataPreProcess(
    imgs, annotations, batchSize, shapeOfInput)
del imgs
del annotations


In [17]:
DLTrain, DLVal = dataLoadersCreation(
    X_train, X_val, y_train, y_val, batchSize)
del X_train
del y_train
del X_val
del y_val


In [18]:
yolov5Model = yolov5ModelCreation(noClasses)
optimizer = optim.Adam(yolov5Model.parameters(), lr=0.001)
yolov5LossFunction= YOLOv5Loss(noClasses=noClasses)
yolov5Model = yolov5Model.to(device)
yolov5LossFunction = yolov5LossFunction.to(device)

Overriding model.yaml nc=80 with nc=47

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]            

In [19]:
trainedModel = TrainModel(yolov5Model, DLTrain, epochs, optimizer, yolov5LossFunction, device)

Epoch 1/100:
0%,0%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 0.8377
Time taken: 0.0min, 51.68099117279053, secs
Epoch 2/100:
0%,0%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 0.0279
Time taken: 0.0min, 42.68564486503601, secs
Epoch 3/100:
0%,0%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 0.0117
Time taken: 0.0min, 42.95397686958313, secs
Epoch 4/100:
0%,0%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 0.0067
Time taken: 0.0min, 42.875529527664185, secs
Epoch 5/100:
0%,0%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 0.0052
Time taken: 0.0min, 42.738624572753906, secs
Epoch 6/100:
0%,0%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 0.0033
Time taken: 0.0min, 42.8049898147583, secs
Epoch 7/100:
0%,0%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 

In [20]:
torch.save(trainedModel.state_dict(), 'trained_yolov5Modelv3.pth')

In [21]:
EvaluateModel(yolov5Model, DLVal, device)


Evaluateing Model:
0%,10%,20%,40%,50%,60%,70%,90%,mAP score on validation set: 0.0000
Time taken: 0.0min, 4.427720546722412, secs
